In [1]:
#установка библиотек
!pip install -U psycopg2
!pip install -U plotly

In [2]:
import psycopg2
psycopg2.__version__

'2.9.3 (dt dec pq3 ext lo64)'

# проверка подключения

In [3]:
# student11
# student11_password

In [4]:
#проверка подключения PostgreSQL 
import psycopg2
import pandas as pd
#Библиотека ждя визуализации
from IPython.display import HTML
import plotly.express as px


#!введите свои реквизиты!
DB_HOST = '52.157.159.24'
DB_USER = 'student11'
DB_USER_PASSWORD = 'student11_password'
DB_NAME = 'sql_ex_third_stream'

conn = psycopg2.connect(host=DB_HOST, user=DB_USER, password=DB_USER_PASSWORD, dbname=DB_NAME)

In [5]:
conn

<connection object at 0x7fbdd2ffa830; dsn: 'user=student11 password=xxx dbname=sql_ex_third_stream host=52.157.159.24', closed: 0>

In [6]:
conn.isexecuting()

False

# задания












## task1
* Корабли: Для каждого класса определите число кораблей этого класса, потопленных в сражениях. Вывести: класс и число потопленных кораблей.












In [7]:
SQL1 =  """
            SELECT classes.class, count(1)
            FROM classes 
            INNER JOIN ships ON classes.class = ships.class 
            INNER JOIN outcomes ON ships.name = outcomes.ship 
            WHERE outcomes.result = 'sunk'
            GROUP BY classes.class
        """

df = pd.read_sql_query(SQL1, conn)
df


,class,count
0,Kongo,1


## task2
* Корабли: Для каждого класса определите год, когда был спущен на воду первый корабль этого класса. Если год спуска на воду головного корабля неизвестен, определите минимальный год спуска на воду кораблей этого класса. Вывести: класс, год.

In [8]:
SQL1 =  """
            SELECT classes.class, MIN(ships.launched)
            FROM classes 
            INNER JOIN ships ON classes.class = ships.class 
            GROUP BY classes.class
        """

df = pd.read_sql_query(SQL1, conn)
df


,class,min
0,Tennessee,1920
1,Revenge,1916
2,Yamato,1941
3,Kongo,1913
4,Renown,1916
5,North Carolina,1941
6,Iowa,1943


## task3
* Корабли: Для классов, имеющих потери в виде потопленных кораблей и не менее 3 кораблей в базе данных, вывести имя класса и число потопленных кораблей.

In [9]:
SQL1 =  """
            WITH have_sunk AS
            (
                SELECT classes.class, count(1) as count_sunk
                FROM classes 
                INNER JOIN ships ON classes.class = ships.class 
                INNER JOIN outcomes ON ships.name = outcomes.ship 
                WHERE outcomes.result = 'sunk'
                GROUP BY classes.class
            ),
            count_ge_3 AS
            (
                SELECT classes.class, count(1)
                FROM classes 
                INNER JOIN ships ON classes.class = ships.class 
                GROUP BY classes.class
            )
            SELECT have_sunk.class, have_sunk.count_sunk
            FROM have_sunk
            INNER JOIN count_ge_3
            ON have_sunk.class = count_ge_3.class
        """

df = pd.read_sql_query(SQL1, conn)
df


,class,count_sunk
0,Kongo,1


## task4
* Корабли: Найдите названия кораблей, имеющих наибольшее число орудий среди всех кораблей такого же водоизмещения (учесть корабли из таблицы Outcomes).

In [10]:
SQL1 =  """
            WITH ships_in_outcomes AS
            (
                SELECT DISTINCT ships.name, ships.class, classes.numguns, classes.displacement
                FROM ships 
                INNER JOIN classes ON ships.class = classes.class 
                INNER JOIN outcomes ON ships.name = outcomes.ship 
            ),
            displacement_numguns AS
            (
                SELECT classes.displacement, MAX(classes.numguns) AS max_numguns
                FROM classes 
                GROUP BY displacement
            ) 
            SELECT ships_in_outcomes.name
            FROM ships_in_outcomes
            INNER JOIN displacement_numguns
            ON ships_in_outcomes.displacement = displacement_numguns.displacement
            WHERE ships_in_outcomes.numguns = displacement_numguns.max_numguns
        """

df = pd.read_sql_query(SQL1, conn)
df


,name
0,California
1,Washington
2,South Dakota
3,Tennessee


## task5
* Компьютерная фирма: Найдите производителей принтеров, которые производят ПК с наименьшим объемом RAM и с самым быстрым процессором среди всех ПК, имеющих наименьший объем RAM. Вывести: Maker

In [20]:
SQL1 =  """
            WITH printer_maker AS
            (
                SELECT DISTINCT maker
                FROM printer
                INNER JOIN product
                ON printer.model = product.model
            ),
            pc_maker AS
            (
                SELECT *
                FROM pc
                INNER JOIN product
                ON pc.model = product.model
            ),
            min_ram_from_pc AS
            (
                SELECT MIN(ram) AS min_ram
                FROM pc
            ),
            max_speed_from_min_ram AS
            (
                SELECT MAX(speed) AS max_speed
                FROM pc
                WHERE ram = (SELECT min_ram FROM min_ram_from_pc)
            )
            SELECT printer_maker.maker
            FROM printer_maker
            INNER JOIN pc_maker
            ON printer_maker.maker = pc_maker.maker
            WHERE (pc_maker.ram = (SELECT min_ram FROM min_ram_from_pc))
                AND (pc_maker.speed = (SELECT max_speed FROM max_speed_from_min_ram))
        """

df = pd.read_sql_query(SQL1, conn)
df


,maker
0,E
1,A
